In [1]:
import numpy as np

%matplotlib notebook
import matplotlib.pyplot as plt

import sys
sys.path.append('..')
from utils import *

DATA_PATH = '../../data/esp/'

%load_ext autoreload
%autoreload 2

In [9]:
df = load_data(DATA_PATH,'train',nclasses=2).loc[:,['review_content','review_rate']]
df.head(5)

Dataset cargado para 2 clases (malo=0, bueno=1)
Num samples per category:
0    184750
1    184750
Name: review_rate, dtype: int64


,review_content,review_rate
0,"Medio berreta, no justifica el gasto, ya se sa...",0
1,Excelente !! superó mi expectativas. Lo único ...,1
2,Es una buena relación calidad/precio. La cámar...,1
3,"Muy poca voluntad para ayudarme con cambiarlo,...",0
4,Nunca me respondieron por el manual de uso. En...,0


In [16]:
from sklearn.feature_extraction.text import CountVectorizer

def vectorize_bow(ds,token_pattern,ngram_range,max_df,min_df,max_features,vocab):
    vec = CountVectorizer(input='content',encoding='utf-8',decode_error='strict',strip_accents=None,
              lowercase=True,preprocessor=None,tokenizer=None,stop_words=None,token_pattern=token_pattern,
              ngram_range=(1,1),analyzer='word',max_df=max_df,min_df=min_df,max_features=max_features,
              vocabulary=vocab,binary=False,dtype=float)
    X = vec.fit_transform(ds)
    return X, vec.vocabulary_
        

X, vocab = vectorize_bow(df['review_content'],token_pattern=r'(\w+|[\.,;:"\'!¡\?¿])',
                ngram_range=(1,1),max_df=1.0,min_df=1,max_features=10000,vocab=None)

X_tfidf = vsm.tfidf(X)
X_ppmi = vsm.ppmi(X)
y = df['review_rate'].values.astype(float)

In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

clf = MultinomialNB()
clf.fit(X,y)
y_pred = clf.predict(X)
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

         0.0       0.91      0.85      0.88    184750
         1.0       0.86      0.92      0.89    184750

    accuracy                           0.88    369500
   macro avg       0.89      0.88      0.88    369500
weighted avg       0.89      0.88      0.88    369500



In [18]:
clf.fit(X_tfidf,y)
y_pred = clf.predict(X_tfidf)
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

         0.0       0.90      0.81      0.85    184750
         1.0       0.83      0.91      0.87    184750

    accuracy                           0.86    369500
   macro avg       0.87      0.86      0.86    369500
weighted avg       0.87      0.86      0.86    369500



In [19]:
clf.fit(X_ppmi,y)
y_pred = clf.predict(X_ppmi)
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

         0.0       0.90      0.85      0.87    184750
         1.0       0.86      0.90      0.88    184750

    accuracy                           0.88    369500
   macro avg       0.88      0.88      0.88    369500
weighted avg       0.88      0.88      0.88    369500

